In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel
from agents.mcp import MCPServerStdio
import subprocess

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [4]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### SYSTEM TOOLS

In [5]:
app_launcher_instruction = """You are an App Launcher. When the user gives a command like "open Chrome" or "launch VS Code", identify the application name and use to provided tool to open it"""

file_searcher_instruction = """You are a File Searcher. When the user asks for a file,use the tool and return the exact file path if it exists. """

clipboard_manager_instruction = """You are a Clipboard Manager. When the user asks to save the clipboard content, use the tool to save it. When the user asks for the clipboard history, use the tool to get it."""


In [7]:
from launch_app import launch_app

In [8]:
APP_LAUNCHER_AGENT=Agent(name="Gemini App laucher", instructions=app_launcher_instruction, model=gemini_model, tools=[launch_app])

In [9]:
with trace("App launcher"):
    result = await Runner.run(APP_LAUNCHER_AGENT, "please open chrome")

✅ Launching chrome...


In [28]:
print(result.final_output)

`open -a "Google Chrome"`


In [10]:
from search_file import search_file

In [74]:
search_file("pdf_data.json")

TypeError: 'FunctionTool' object is not callable

In [11]:
FILE_SEARCHER_AGENT=Agent(name="Gemini Folder/File searcher", instructions=file_searcher_instruction, model="gpt-4o-mini", tools=[search_file])

In [12]:
with trace("File searcher"):
    result = await Runner.run(FILE_SEARCHER_AGENT, "Can you please tell me where is pdf_data.json file")

In [13]:
print(result.final_output)

The file `pdf_data.json` is located at:

```
C:\Users\talha\Desktop\LLM\llmassignment\law\pdf_data.json
```


In [6]:
from clipboard_manager import save_clipboard, get_clipboard_history, get_last_clipboard

In [7]:
CLIPBOARD_MANAGER=Agent(name="Gemini  clipboard manager", instructions=clipboard_manager_instruction, model=gemini_model, tools=[save_clipboard, get_clipboard_history, get_last_clipboard])

In [9]:
with trace("clipboard manager"):
    result = await Runner.run(CLIPBOARD_MANAGER, "can you please save my clip board history, and show it to me")

✅ Saved to history: [project]
name = "agents"
version = "0.1.0"
des...


In [10]:
with trace("clipboard manager"):
    result = await Runner.run(CLIPBOARD_MANAGER, "can you please save my clip board history, and show it to me")

✅ Saved to history: # The imports

from dotenv import load_dotenv
f...


### INTERNET TOOLS

In [14]:
playwrite_instructions="You are internet searcher agent, your job is to use playwrite tool and search the relevent query and perform actions to give best answer to user"

In [17]:
# The imports
from agents.mcp import MCPServerStdio


In [18]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

Error initializing MCP server: fileno


UnsupportedOperation: fileno

In [15]:
PLAYWRITE_AGENT=Agent(name="GPT PLAYWRITE", instructions=playwrite_instructions, model="gpt-4o-mini", tools=tools)

In [16]:
with trace("searcher manager"):
    result = await Runner.run(PLAYWRITE_AGENT, "can you please tell me what is the weather in pakistan ,punjab ,lahore ,today")

Error getting response: Unknown tool type: <class 'langchain_community.tools.playwright.click.ClickTool'>, tool. (request_id: None)


UserError: Unknown tool type: <class 'langchain_community.tools.playwright.click.ClickTool'>, tool